In [3]:
#=
Autor: Edvan S. Sousa
Ano: dezembro 2020
Problema original: 
Versão Julia:

FE4:
1) Mpi Hello
2) Mpi Hello com comunicacao
3) Mpi Fatorial

FE4 SOL:
4) Mpi Fatorial com comunicacao
5) Mpi Fatorial com comunicacao parametrizado

FE5:
6) Mpi Trapézio v2
7) Mpi Trapézio v3

FE5 SOL:
8) Mpi Simpson Reduction

FE6:
9) Mpi Mult Matriz x Vetor
10) Mpi Odd Even

11) Questao 01
12) Questao 04 1
13) Questao 04 2
14) Questão 05
https://hendrikdcomp.bitbucket.io/ppc/sala/
=#

In [1]:
# Questão 01
# MPI-Hello
import MPI

function main()
MPI.Init()
comm = MPI.COMM_WORLD

print("Hello world, I am rank $(MPI.Comm_rank(comm)) of $(MPI.Comm_size(comm))\n")
MPI.Barrier(comm)

MPI.Finalize()# Opcional. Invocado automaticamente qdo Julia encerra. 
end

main()

Hello world, I am rank 0 of 1


In [1]:
# Questão 02
# MPI-Hello-comunicação
import MPI

function main()
    
MPI.Init()
comm = MPI.COMM_WORLD
greeting = ""
    
comm_sz = MPI.Comm_size(comm)
my_rank = MPI.Comm_rank(comm)
root = 0
dst = mod(my_rank + 1, comm_sz)
src = mod(my_rank-1, comm_sz)
tag = 9

if my_rank != root
    # Create message
    greeting = "Greetings from process $(my_rank) of $(comm_sz)!"
    # Send message to process 0 
    # Descrição Isend(data, dest::Integer, tag::Integer, comm::Comm)
    MPI.Isend(greeting, dst, tag, comm)
else
    #Print my message
    println("Greeting@s from process $(my_rank) of $(comm_sz)!")
    for q in 1:comm_sz
        #Receive message from process q
        # Descrição Recv!(data, src::Integer, tag::Integer, comm::Comm)
        MPI.Irecv!(greeting, q, tag, comm)
        #Print message from process q */
        println("$(greeting)")
     end
end

MPI.Barrier(comm)
MPI.Finalize()
end

main()


Greeting@s from process 0 of 1!


LoadError: MethodError: no method matching MPI.Buffer(::String)
Closest candidates are:
  MPI.Buffer(::A, !Matched::Int32, !Matched::MPI.Datatype) where A at C:\Users\sousa\.julia\packages\MPI\b7MVG\src\buffers.jl:123
  MPI.Buffer(::Any, !Matched::Integer, !Matched::MPI.Datatype) at C:\Users\sousa\.julia\packages\MPI\b7MVG\src\buffers.jl:135
  MPI.Buffer(!Matched::MPI.Buffer) at C:\Users\sousa\.julia\packages\MPI\b7MVG\src\buffers.jl:134
  ...

In [ ]:
# examples/04-sendrecv.jl
using MPI
MPI.Init()

comm = MPI.COMM_WORLD
rank = MPI.Comm_rank(comm)
size = MPI.Comm_size(comm)

dst = mod(rank+1, size)
src = mod(rank-1, size)

N = 4

send_mesg = Array{Float64}(undef, N)
recv_mesg = Array{Float64}(undef, N)

fill!(send_mesg, Float64(rank))

rreq = MPI.Irecv!(recv_mesg, src,  src+32, comm)

print("$rank: Sending   $rank -> $dst = $send_mesg\n")
sreq = MPI.Isend(send_mesg, dst, rank+32, comm)

stats = MPI.Waitall!([rreq, sreq])

print("$rank: Received $src -> $rank = $recv_mesg\n")

MPI.Barrier(comm)


In [1]:
#https://ddd.uab.cat/pub/artpub/2016//174327/comeco_a2016v48p535iENG_postprint.pdf
import MPI
import Statistics

function main()
    
    MPI.Init()
    comm = MPI.COMM_WORLD
    size = MPI.Comm_size(comm)
    rank = MPI.Comm_rank(comm)
    
    recv_mesg = zeros(100, 1)
    
    if rank == 0
        send_mesg = rand(100, 1)
        for i = 1:size-1
            MPI.Send(send_mesg, i, 0, comm)
        end
    else
        MPI.Recv!(recv_mesg, 0, 0, comm)
    end
    
    m = Statistics.mean(+, recv_mesg)
    sleep(rank*2)
    println("Results: rank: ", rank, " -> média: ", m)
    
    MPI.Barrier(comm)
    MPI.Finalize()
end

main()

Results: rank: 0 -> média: 0.0
